In [17]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

min_value = 2
max_value = 7
hour = 15
work_hours = 9
employee_cost = 80

requirement = np.random.randint(min_value, max_value + 1, size=hour)

num_employees = 30

model = gp.Model('Shift Scheduling')

work = {}
overnight = {}

for i in range(num_employees):
    overnight[i] = model.addVar(vtype=GRB.BINARY, name=f'employee_{i}_works_overnight')
    for j in range(hour - work_hours + 1):
        for k in range(j, j + work_hours):
            work[(i, j, k)] = model.addVar(vtype=GRB.BINARY, name=f'employee_{i}_start_{j}_work_{k}')

# Constraints
for i in range(num_employees):
    for j in range(hour - work_hours + 1):
        model.addConstr(gp.quicksum(work[(i, j, k)] for k in range(j, j + work_hours)) == 8 * work[(i, j, j)])

for i in range(num_employees):
    model.addConstr(gp.quicksum(work[(i, j, j)] for j in range(hour - work_hours + 1)) <= 1)

for i in range(num_employees):
    for j in range(6):
        for k in range(j, 6):
            model.addConstr(overnight[i] >= work[(i, j, k)])

for i in range(num_employees):
    for j in range(hour - work_hours + 1):
        model.addConstr(gp.quicksum(work[(i, j, k)] for k in range(j + 3, j + 6)) == 2 * work[(i, j, j)])

for k in range(hour):
    model.addConstr(gp.quicksum(work[(i, j, k)] for i in range(num_employees)
                                for j in range(max(0, k - work_hours + 1), k + 1)
                                if (i, j, k) in work) >= requirement[k])

# Objective Function
model.setObjective(10 * (gp.quicksum(work[(i, j, k)] for i in range(num_employees)
                                     for j in range(hour - work_hours + 1)
                                     for k in range(j, j + work_hours)))
                   + 20 * (gp.quicksum(overnight[i] for i in range(num_employees))),
                   GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Check if the solution is optimal
if model.status == GRB.OPTIMAL:
    print(f'Optimal objective value (total cost): {model.objVal}')
    
    for i in range(num_employees):
        for j in range(hour - work_hours + 1):
            if work[(i, j, j)].x > 0.5:
                for k in range(j, j + work_hours):
                    if work[(i, j, k)].x < 0.5:
                        print(f'Employee {i} starts at hour {j} and takes a break at hour {k}')
                    if overnight[i].x>0.5:
                        print(f'employee {i} works overnight ')

    
    # arr = []
    # for k in range(hour):
    #     sumi = 0
    #     for j in range(max(0, k - work_hours + 1), k + 1):
    #         for i in range(num_employees):
    #             if (i, j, k) in work:  
    #                 sumi += int(work[(i, j, k)].x)
    #     arr.append(sumi)
    
    # print("Employees working each hour:", arr)
    # print("Hourly requirements:", requirement)
else:
    print('No optimal solution found.')


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1095 rows, 1920 columns and 6090 nonzeros
Model fingerprint: 0xf3f27161
Variable types: 0 continuous, 1920 integer (1920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 630 rows and 1050 columns
Presolve time: 0.02s
Presolved: 465 rows, 870 columns, 3510 nonzeros
Variable types: 0 continuous, 870 integer (870 binary)
Found heuristic solution: objective 2360.0000000
Found heuristic solution: objective 1960.0000000

Root relaxation: objective 1.260000e+03, 724 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Ob

In [18]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import csv

min_value = 2
max_value = 7
hour = 15
work_hours = 9
employee_cost = 80

requirement = np.random.randint(min_value, max_value + 1, size=hour)

num_employees = 30

model = gp.Model('Shift Scheduling')

work = {}
overnight = {}

for i in range(num_employees):
    overnight[i] = model.addVar(vtype=GRB.BINARY, name=f'employee_{i}_works_overnight')
    for j in range(hour - work_hours + 1):
        for k in range(j, j + work_hours):
            work[(i, j, k)] = model.addVar(vtype=GRB.BINARY, name=f'employee_{i}_start_{j}_work_{k}')

# Constraints
for i in range(num_employees):
    for j in range(hour - work_hours + 1):
        model.addConstr(gp.quicksum(work[(i, j, k)] for k in range(j, j + work_hours)) == 8 * work[(i, j, j)])

for i in range(num_employees):
    model.addConstr(gp.quicksum(work[(i, j, j)] for j in range(hour - work_hours + 1)) <= 1)

for i in range(num_employees):
    for j in range(6):
        for k in range(j, 6):
            model.addConstr(overnight[i] >= work[(i, j, k)])
            #model.addConstr(overnight[i] ==0)

for i in range(num_employees):
    for j in range(hour - work_hours + 1):
        model.addConstr(gp.quicksum(work[(i, j, k)] for k in range(j + 3, j + 6)) == 2 * work[(i, j, j)])

for k in range(hour):
    model.addConstr(gp.quicksum(work[(i, j, k)] for i in range(num_employees)
                                for j in range(max(0, k - work_hours + 1), k + 1)
                                if (i, j, k) in work) >= requirement[k])

# Objective Function
model.setObjective(10 * (gp.quicksum(work[(i, j, k)] for i in range(num_employees)
                                     for j in range(hour - work_hours + 1)
                                     for k in range(j, j + work_hours)))
                   + 20 * (gp.quicksum(overnight[i] for i in range(num_employees))),
                   GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Check if the solution is optimal
if model.status == GRB.OPTIMAL:
    print(f'Optimal objective value (total cost): {model.objVal}')
    
    # Prepare data for CSV
    with open('shift_schedule_output.csv', 'w', newline='') as csvfile:
        fieldnames = ['Employee', 'Start Hour', 'lunch Hour', 'Overnight']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(num_employees):
            overnight_flag = 1 if overnight[i].x > 0.5 else 0
            for j in range(hour - work_hours + 1):
                if work[(i, j, j)].x > 0.5:
                    for k in range(j, j + work_hours):
                        if work[(i, j, k)].x < 0.5:
                            writer.writerow({
                            'Employee': i,
                            'Start Hour': j,
                            'lunch Hour': k,
                            'Overnight': overnight_flag
                        })
    
    print('Output saved to shift_schedule_output.csv')
else:
    print('No optimal solution found.')


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1095 rows, 1920 columns and 6090 nonzeros
Model fingerprint: 0xa5b1bda8
Variable types: 0 continuous, 1920 integer (1920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 630 rows and 1050 columns
Presolve time: 0.00s
Presolved: 465 rows, 870 columns, 3510 nonzeros
Variable types: 0 continuous, 870 integer (870 binary)
Found heuristic solution: objective 1520.0000000

Root relaxation: objective 1.170000e+03, 657 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/